In [1]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [2]:
dict_tablas = pickle.load(open('dict_tablas','rb'))
df_historical_data = pd.read_csv(r'Datasets\Clean_Fifa_WorldCup_Matches.csv')
df_fixture = pd.read_csv(r'Datasets\Fifa_WorldCup_Fixture.csv')

In [3]:
dict_tablas['Grupo F']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Belgium,0,0,0,0,0,0,0,0
1,2,Canada,0,0,0,0,0,0,0,0
2,3,Morocco,0,0,0,0,0,0,0,0
3,4,Croatia,0,0,0,0,0,0,0,0


## 1.- Calcular Nivel de equipo

In [4]:
#Dividir df en df_home y df_away
df_historical_data


,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals
0,France,Mexico,1930,4,1
1,Argentina,France,1930,1,0
2,Chile,Mexico,1930,3,0
3,Chile,France,1930,1,0
4,Argentina,Mexico,1930,6,3
...,...,...,...,...,...
895,Russia,Croatia,2018,2,2
896,France,Belgium,2018,1,0
897,Croatia,England,2018,2,1
898,Belgium,England,2018,2,0


In [5]:
#Dividir df en df_home y df_away
df_home = df_historical_data[['HomeTeam','HomeGoals','AwayGoals']]
df_away = df_historical_data[['AwayTeam','HomeGoals','AwayGoals']]

In [6]:
#Renombrar Columnas
df_home = df_home.rename(columns={'HomeTeam':'Team','HomeGoals':'GoalScored','AwayGoals':'GoalConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team','HomeGoals':'GoalConceded','AwayGoals':'GoalScored'})

In [7]:
#Concatenar df_home y df_away, hacer grupo por team y calcular promedio
df_team_strenght = pd.concat([df_home, df_away], ignore_index=True).groupby('Team').mean()

#Convertir la primera letra a minúscula
df_team_strenght.info()
df_team_strenght.head()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, Algeria to Zaire
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   GoalScored    85 non-null     float64
 1   GoalConceded  85 non-null     float64
dtypes: float64(2)
memory usage: 2.0+ KB


,GoalScored,GoalConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690


In [8]:
def prediction_points(home,away):
    if home in df_team_strenght.index and away in df_team_strenght.index:
        #Para obtener lambda vamos a multiplicar el promedio de goles anotas del
        #equipo rival por el promedio de goles recibidos del equipo visitante
        #goals_scored * goals_conceded
        lamb_home = df_team_strenght.at[home, 'GoalScored'] * df_team_strenght.at[away,'GoalConceded']
        lamb_away = df_team_strenght.at[away, 'GoalScored'] * df_team_strenght.at[home,'GoalConceded']
        prob_home, prob_away, prob_draw = 0,0,0
        for x in range(0,11): #Number of goals home team
            for y in range(0,11): #Number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0,0)

## 2.1- Probamos nuestra función

In [9]:
#Testear con partidos: Argentina - México, England - United States, Qatar(H) -Ecuador
prediction_points('argentina','mexico')
prediction_points('england','united states')
#prediction_points('Qatar','Ecuador')

(0, 0)

## 3.- Prediciendo Resultados de Mundial

In [10]:
#Dividiendo fixture en grupo, octavos, cuartos,...
df_fixture_gruop_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semis = df_fixture[60:62].copy()
df_fixture_third_place = df_fixture[62:63].copy()
df_fixture_final = df_fixture[63:].copy()

In [11]:
#Correr todos los partidos de la fase de grupos y actualizamos las tablas de cada grupo
for group in dict_tablas:
    print(f"Procesando grupo: {group}")
    teams_in_group = dict_tablas[group]['Team'].values
    print(f"Equipos en el grupo: {teams_in_group}")
    
    # Verificar contenido de teams_in_group
    print("Contenido de teams_in_group:")
    for team in teams_in_group:
        print(f"'{team}'")
    
    teams_in_group = [team.strip().lower() for team in dict_tablas[group]['Team'].values]
    df_fixture_gruop_48['Home'] = df_fixture_gruop_48['Home'].str.strip().str.lower()
    df_fixture_gruop_48['Away'] = df_fixture_gruop_48['Away'].str.strip().str.lower()

    df_fixture_gruop_6 = df_fixture_gruop_48[df_fixture_gruop_48['Home'].isin(teams_in_group)]
    print(f"Fixture filtrado para el grupo: {df_fixture_gruop_6}")
    
    if df_fixture_gruop_6.empty:
        print(f"No hay partidos para los equipos en el grupo: {group}")
        continue
    
    for index, row in df_fixture_gruop_6.iterrows():
        home, away = row['Home'].capitalize(), row['Away'].capitalize()
        print(f"Partido: {home} vs {away}")
        
        points_home, points_away = prediction_points(home, away)
        print(f"Puntos predichos - Home: {points_home}, Away: {points_away}")
        
        dict_tablas[group].loc[dict_tablas[group]['Team'] == home, 'Pts'] += points_home
        dict_tablas[group].loc[dict_tablas[group]['Team'] == away, 'Pts'] += points_away
    
    dict_tablas[group] = dict_tablas[group].sort_values('Pts', ascending=False).reset_index(drop=True)
    dict_tablas[group] = dict_tablas[group][['Team','Pts']]
    dict_tablas[group] = dict_tablas[group].round(0)
    #print(f"Tabla actualizada para el grupo {group}:")
    print(dict_tablas[group])

Procesando grupo: Grupo A
Equipos en el grupo: ['Qatar (H)' 'Ecuador' 'Senegal' 'Netherlands']
Contenido de teams_in_group:
'Qatar (H)'
'Ecuador'
'Senegal'
'Netherlands'
Fixture filtrado para el grupo:           Home     Score         Away  Year
1      senegal   Match 2  netherlands  2022
3  netherlands  Match 19      ecuador  2022
4      ecuador  Match 35      senegal  2022
5  netherlands  Match 36        qatar  2022
Partido: Senegal vs Netherlands
Puntos predichos - Home: 0.8966936039774208, Away: 1.8974813628251535
Partido: Netherlands vs Ecuador
Puntos predichos - Home: 1.996910350307241, Away: 0.7820971005136241
Partido: Ecuador vs Senegal
Puntos predichos - Home: 1.1947181862346672, Away: 1.553002815761932
Partido: Netherlands vs Qatar
Puntos predichos - Home: 0, Away: 0
          Team  Pts
0  Netherlands  4.0
1      Senegal  2.0
2      Ecuador  2.0
3    Qatar (H)  0.0
Procesando grupo: Grupo B
Equipos en el grupo: ['England' 'Iran' 'United States' 'Wales']
Contenido de teams_in_

C:\Users\Sam Contreras\AppData\Local\Temp\ipykernel_16340\3621081397.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.8966936]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_tablas[group].loc[dict_tablas[group]['Team'] == home, 'Pts'] += points_home
C:\Users\Sam Contreras\AppData\Local\Temp\ipykernel_16340\3621081397.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.38825101]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_tablas[group].loc[dict_tablas[group]['Team'] == home, 'Pts'] += points_home


Puntos predichos - Home: 1.9925693430207814, Away: 0.7122928684600978
Partido: England vs United states
Puntos predichos - Home: 0, Away: 0
Partido: Wales vs England
Puntos predichos - Home: 1.0828229517246506, Away: 1.593887353091192
Partido: Iran vs United states
Puntos predichos - Home: 0, Away: 0
            Team  Pts
0        England  4.0
1          Wales  3.0
2           Iran  1.0
3  United States  0.0
Procesando grupo: Grupo C
Equipos en el grupo: ['Argentina' 'Saudi Arabia' 'Mexico' 'Poland']
Contenido de teams_in_group:
'Argentina'
'Saudi Arabia'
'Mexico'
'Poland'
Fixture filtrado para el grupo:             Home     Score          Away  Year
12     argentina   Match 8  saudi arabia  2022
13        mexico   Match 7        poland  2022
14        poland  Match 22  saudi arabia  2022
15     argentina  Match 24        mexico  2022
16        poland  Match 39     argentina  2022
17  saudi arabia  Match 40        mexico  2022
Partido: Argentina vs Saudi arabia
Puntos predichos - Home:

C:\Users\Sam Contreras\AppData\Local\Temp\ipykernel_16340\3621081397.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.86042645]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_tablas[group].loc[dict_tablas[group]['Team'] == home, 'Pts'] += points_home
C:\Users\Sam Contreras\AppData\Local\Temp\ipykernel_16340\3621081397.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.1926621]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_tablas[group].loc[dict_tablas[group]['Team'] == home, 'Pts'] += points_home


Puntos predichos - Home: 0.4642855217418519, Away: 2.3942534773618096
Partido: Tunisia vs France
Puntos predichos - Home: 0.40992988425551813, Away: 2.4593706967067446
        Team  Pts
0     France  7.0
1    Denmark  6.0
2    Tunisia  3.0
3  Australia  2.0
Procesando grupo: Grupo E
Equipos en el grupo: ['Spain' 'Costa Rica' 'Germany' 'Japan']
Contenido de teams_in_group:
'Spain'
'Costa Rica'
'Germany'
'Japan'
Fixture filtrado para el grupo:           Home     Score        Away  Year
24     germany  Match 11       japan  2022
25       spain  Match 10  costa rica  2022
26       japan  Match 25  costa rica  2022
27       spain  Match 28     germany  2022
28       japan  Match 43       spain  2022
29  costa rica  Match 44     germany  2022
Partido: Germany vs Japan
Puntos predichos - Home: 2.398798931491757, Away: 0.4564917304513104
Partido: Spain vs Costa rica
Puntos predichos - Home: 0, Away: 0
Partido: Japan vs Costa rica
Puntos predichos - Home: 0, Away: 0
Partido: Spain vs Germany
Pu

C:\Users\Sam Contreras\AppData\Local\Temp\ipykernel_16340\3621081397.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.39879893]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_tablas[group].loc[dict_tablas[group]['Team'] == home, 'Pts'] += points_home
C:\Users\Sam Contreras\AppData\Local\Temp\ipykernel_16340\3621081397.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.71189919]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_tablas[group].loc[dict_tablas[group]['Team'] == home, 'Pts'] += points_home


Puntos predichos - Home: 1.787735286577132, Away: 0.9920917383543204
Partido: Croatia vs Canada
Puntos predichos - Home: 2.8414620816302367, Away: 0.07916413813302248
Partido: Croatia vs Belgium
Puntos predichos - Home: 1.7935719490022932, Away: 1.006567821071216
Partido: Canada vs Morocco
Puntos predichos - Home: 0.23262365791729264, Away: 2.534751423201758
      Team  Pts
0  Croatia  7.0
1  Belgium  6.0
2  Morocco  4.0
3   Canada  0.0
Procesando grupo: Grupo G
Equipos en el grupo: ['Brazil' 'Serbia' 'Switzerland' 'Cameroon']
Contenido de teams_in_group:
'Brazil'
'Serbia'
'Switzerland'
'Cameroon'
Fixture filtrado para el grupo:            Home     Score         Away  Year
36  switzerland  Match 13     cameroon  2022
37       brazil  Match 16       serbia  2022
38     cameroon  Match 29       serbia  2022
39       brazil  Match 31  switzerland  2022
40       serbia  Match 47  switzerland  2022
41     cameroon  Match 48       brazil  2022
Partido: Switzerland vs Cameroon
Puntos predicho

C:\Users\Sam Contreras\AppData\Local\Temp\ipykernel_16340\3621081397.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.06786741]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_tablas[group].loc[dict_tablas[group]['Team'] == home, 'Pts'] += points_home
C:\Users\Sam Contreras\AppData\Local\Temp\ipykernel_16340\3621081397.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.94865532]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_tablas[group].loc[dict_tablas[group]['Team'] == home, 'Pts'] += points_home


Puntos predichos - Home: 2.5007861849138986, Away: 0.38500999130256797
Partido: Serbia vs Switzerland
Puntos predichos - Home: 1.085852480261606, Away: 1.6641775568977955
Partido: Cameroon vs Brazil
Puntos predichos - Home: 0.16070278218467365, Away: 2.7667880407905994
          Team  Pts
0       Brazil  8.0
1  Switzerland  4.0
2       Serbia  3.0
3     Cameroon  2.0
Procesando grupo: Grupo H
Equipos en el grupo: ['Portugal' 'Ghana' 'Uruguay' 'South Korea']
Contenido de teams_in_group:
'Portugal'
'Ghana'
'Uruguay'
'South Korea'
Fixture filtrado para el grupo:            Home     Score         Away  Year
42      uruguay  Match 14  south korea  2022
43     portugal  Match 15        ghana  2022
44  south korea  Match 30        ghana  2022
45     portugal  Match 32      uruguay  2022
46        ghana  Match 45      uruguay  2022
47  south korea  Match 46     portugal  2022
Partido: Uruguay vs South korea
Puntos predichos - Home: 0, Away: 0
Partido: Portugal vs Ghana
Puntos predichos - Home:

In [18]:
#mostrar tabla actualizada
dict_tablas['Grupo H']

,Team,Pts
0,Portugal,4.0
1,Uruguay,3.0
2,Ghana,2.0
3,South Korea,0.0
